In [ ]:
!rm -rf iFeature
!git clone https://github.com/Superzchen/iFeature.git

Cloning into 'iFeature'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 322 (delta 40), reused 33 (delta 29), pack-reused 275
Receiving objects: 100% (322/322), 6.72 MiB | 21.45 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

biodata_txt = '/content/drive/My Drive/Testdata/biodata.txt'


# read txt file
# ----------------------------------------------------
"""
# Load and get label in data : biodata_txt
with open(biodata_txt, 'r') as file:
    data = [line.split() for line in file.read().splitlines()]

# Split sequences and labels
GeneNames = [item[0] for item in data]
proteinIDs = [item[1] for item in data]
sequences = [item[2] for item in data]
labels = [1 if item[3] == 'Stable' else 0 for item in data]
"""

# txt file 읽기
# ----------------------------------------------------
file = open(biodata_txt, "r")
data = file.read()
data_split = data.split()

list_Amino = []
list_Stable = []

for i in range(0, len(data_split)):
    if i%4 == 2:
        list_Amino.append(data_split[i])
    if i%4 == 3:
        list_Stable.append(data_split[i])


# 인덱스 지우기
del list_Amino[0]
del list_Stable[0]


# 뒤집은 데이터 추가
# ----------------------------------------------------
# for AC in range(len(list_Amino)) :
#   list_Amino.append(list_Amino[AC][::-1])

# for stability in range(len(list_Stable)) :
#   list_Stable.append(list_Stable[stability])

# ----------------------------------------------------



Mounted at /content/drive


특징 추가 Model (iFeature **csv**)

In [ ]:
import csv

AAC_features_csv = '/content/drive/My Drive/Testdata/AAC_features_model.csv'
CTDC_features_csv = '/content/drive/My Drive/Testdata/CTDC_features_model.csv'

model_feature_files = [ AAC_features_csv, CTDC_features_csv]

model_features_list = []

for feature_file in model_feature_files:
    model_feature_list = []

    # Open the file
    with open(feature_file, 'r') as file:
        # Create CSV reader object
        csv_reader = csv.reader(file, delimiter='\t')
        next(csv_reader)

        # Add each row to the list
        for row in csv_reader:
            model_feature_list.append([float(x) for x in row[1:]])

    model_features_list.append(model_feature_list)

모델 : NNC

In [ ]:
!pip install matplotlib scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd

import csv
import matplotlib.pyplot as plt
import statistics

import os

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# 아미노산 서열의 최대 길이 뽑기
maxlen = 0
for str in list_Amino:
    if maxlen < len(str):
        print(str)
        maxlen = len(str)
print(maxlen)

maxlen = 1653

# 정규화 : 모두 길이를 일정하게 맞춤
for i in range(0, len(list_Amino)):
    while maxlen > len(list_Amino[i]):
        list_Amino[i] += '0'

# ---------------------------------------
xdot = []
x = []
count = 0
for st in list_Amino:
    for letter in st:
        xinit = ord(letter) - 64
        if xinit == -16:
            xinit = 0

        xinit = float(xinit)/26
        xdot.append(xinit)
    x.append(xdot)
    xdot = []
    count += 1
    if count % 100 == 0:
        print(count)

x = np.array(x)
x = torch.tensor(x)

# ---------------------------------------

y = []
for st in list_Stable:
    if st == "Stable":
        y.append(1)
    else:
        y.append(0)
y = torch.tensor(y)


x = x.cuda()
y = y.cuda()

x = x.float()
y = y.int()

file.close()

# features 변수에 저장된 넘파이 배열을 이용하여 모델 학습

# Preprocessing and data loading part would be the same until features loading
# IFeature
# ---------------------------------------
# features를 numpy array로 변환하고, torch tensor로 변환


# AAC features를 numpy array로 변환하고, torch tensor로 변환
AAC_features = np.array(model_features_list[0]).astype(np.float)
AAC_features = torch.tensor(AAC_features).float().to(device)

# CTDC features를 numpy array로 변환하고, torch tensor로 변환
CTDC_features = np.array(model_features_list[1]).astype(np.float)
CTDC_features = torch.tensor(CTDC_features).float().to(device)

# # Concatenate the original features with AAC and CTDC features
x = torch.cat((x, AAC_features), dim=1)
x = torch.cat((x, CTDC_features), dim=1)

inputsize = maxlen + AAC_features.shape[1] + CTDC_features.shape[1]

# inputsize = 1653
# ---------------------------------------

class NNC(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputsize, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for inn, (train, label) in enumerate(dataloader):

        optimizer.zero_grad()
        # 예측(prediction)과 손실(loss) 계산
        pred = model(train)

        label = label.unsqueeze(1)
        label = label.float()

        sig = nn.Sigmoid()

        pred = sig(pred)

        loss = loss_fn(pred, label)

        loss.backward()
        optimizer.step()

        acc = []

        for ien in range(len(label)):
            output = 0
            if pred[ien] > 0.5:
                output = 1
            else:
                output = 0
            if label[ien] == output:
                acc.append(1)
            else:
                acc.append(0)

        acc = sum(acc) / len(acc)

        if inn % 200 == 0:

            loss, current = loss.item(), (inn + 1) * len(train)
            print(f"loss: {loss:>7f} acc: {acc:>5f} [{current:>5d}/{size:>5d}]")


AAC_features_test_csv = '/content/drive/My Drive/Testdata/AAC_features_test.csv'
CTDC_features_test_csv = '/content/drive/My Drive/Testdata/CTDC_features_test.csv'

test_feature_files = [ AAC_features_test_csv, CTDC_features_test_csv ]
# CTDC_features_test_csv
# test_feature_files = []

test_features_list = []
for feature_file in test_feature_files:
    test_feature_list = []

    # Open the file
    with open(feature_file, 'r') as file:
        # Create CSV reader object
        csv_reader = csv.reader(file, delimiter='\t')
        next(csv_reader)

        # Add each row to the list
        for row in csv_reader:
            test_feature_list.append(row[1:])

    test_features_list.append(test_feature_list)


# test_model 함수 정의
def test_model(test_sequences, features_list, ground_truth_labels):
    # Preprocessing the sequences similar to the training sequences
    xdott = []
    x_test = []  # Initialize x_test here
    for i in range(len(test_sequences)):
        st = test_sequences[i]
        for letter in st:
            xinit = ord(letter) - 64
            if xinit == -16:
                xinit = 0
            xinit = float(xinit)/26
            xdott.append(xinit)
        # Append features for the sequence from each feature list
        for feature in features_list:
            feature_values = [float(val) for val in feature[i]]
            xdott.extend(feature_values)
        x_test.append(xdott)
        xdott = []

    # Convert the list to a tensor
    x_test = torch.tensor(x_test).float().to(device)

    y_test = []
    for st in ground_truth_labels:
        if st == "Stable":
            y_test.append(1)
        else:
            y_test.append(0)
    y_test = torch.tensor(y_test).int().to(device)

    # TensorDataset과 DataLoader로 변환합니다.
    test_data = TensorDataset(x_test, y_test)
    test_dataloader = DataLoader(test_data, batch_size=50)  # 적절한 배치 크기를 선택합니다.

    # 예측 결과를 저장할 리스트를 초기화합니다.
    all_probabilities = []

    # 양성 클래스를 1, 음성 클래스를 0으로 매핑한 ground truth labels 생성
    binary_labels = [1 if label == 'Stable' else 0 for label in ground_truth_labels]

    with torch.no_grad():
        mymodel.eval()
        for i, (inputs, labels) in enumerate(test_dataloader):
            output = mymodel(inputs)
            sig = nn.Sigmoid()
            output = sig(output)

            # 예측 확률을 리스트에 추가합니다.
            batch_probabilities = [val[0] for val in output.tolist()]
            all_probabilities.extend(batch_probabilities)

    # 확률의 중간값을 계산합니다.
    median_prob = np.median(all_probabilities)

    print("===================================")
    print("Median Probability:", median_prob)

    # ROC 곡선을 기반으로 임계값을 설정합니다.
    fpr, tpr, thresholds = roc_curve(binary_labels, all_probabilities)
    threshold_idx = np.argmax(tpr - fpr)
    threshold = thresholds[threshold_idx]

    #threshold = median_prob
    print(threshold)

    # 예측 결과를 임계값을 기준으로 설정하여 리스트에 추가합니다.
    all_predictions = ["Stable" if val > threshold else "Degradable" for val in all_probabilities]

    # 정확도 계산
    correct_predictions = sum([1 if prediction == truth else 0 for prediction, truth in zip(all_predictions, ground_truth_labels)])
    accuracy = correct_predictions / len(ground_truth_labels)

    # TPR 및 FPR 계산
    fpr, tpr, thresholds = roc_curve(binary_labels, all_probabilities)

    # AUC 계산
    auc_score = auc(fpr, tpr)

    print("Accuracy:", accuracy)
    print("AUC:", auc_score)

    return all_predictions, accuracy, all_probabilities, y_test

# 드라이브 마운트
drive.mount('/content/drive')
testdata_txt = '/content/drive/My Drive/Testdata/testdata.txt'

# test data 파일 읽기
file = open(testdata_txt, "r")
data = file.read()
data_split = data.split()

test_sequences = []
ground_truth_labels = []

for i in range(0, len(data_split)):
    if i%4 == 2:
        test_sequences.append(data_split[i])
    if i%4 == 3:
        ground_truth_labels.append(data_split[i])

# 인덱스 삭제
del test_sequences[0]
del ground_truth_labels[0]

# 시퀀스 길이 정규화
for i in range(0, len(test_sequences)):
    while maxlen > len(test_sequences[i]):
        test_sequences[i] += '0'

# -------------------model--------------------

mediandata_dir = "median_data"
finaldata_dir = "final_data"

predict10 = []
prob10 = []

for i in range(31) :

    mymodel = NNC().to(device)
    print(len(x))

    learning_rate = 0.0015
    batchsize = 400
    epochs = 429

    dataset = TensorDataset(x,y)
    train_dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)

    loss_func = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(mymodel.parameters(), lr=learning_rate, weight_decay=0.01)


    for t in range(epochs):
        print(f"Epoch {t+1}\n---------------------------")
        train_loop(train_dataloader, mymodel, loss_func, optimizer)

    print("Done")


    # -----------------------------------------------------
    # ----- test

    # 모델 예측 수행
    predictions, accuracy, probabilities, y_test = test_model(test_sequences, test_features_list, ground_truth_labels)

    predict10.append(predictions)
    prob10.append(probabilities)


    # probabilities를 Tensor로 변환합니다.
    probabilities = torch.tensor(probabilities)

    # ROC curve를 계산합니다.
    fpr, tpr, thresholds = roc_curve(y_test.cpu().numpy(), probabilities)

    # AUC를 계산합니다.
    auc_score = auc(fpr, tpr)

    # AUC 그래프를 그립니다.
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

    print("test_sequences: ", test_sequences)
    print("ground_truth_labels: ", ground_truth_labels)

    # 예측 결과와 정확도 출력
    print("Predictions: ", predictions)
    print("Accuracy: ", accuracy)
    print("probabilities: ", probabilities)

    # 텍스트 파일에 예측 결과 저장
    file_path = f'/content/drive/My Drive/Testdata/{mediandata_dir}/test_result_nnc{i}.txt'  # 원하는 경로 및 파일명으로 변경 가능

    with open(file_path, 'w') as file:
        for item in predictions:
            file.write(item + '\n')


print("======================final======================")
from collections import Counter

# 각 시퀀스에 대해 가장 많이 예측된 값을 최종 예측으로 사용합니다.
final_predictions = [statistics.mode([predictions[i] for predictions in predict10]) for i in range(len(test_sequences))]

# 최빈값
mode_counts = [Counter([predictions[i] for predictions in predict10]).most_common(1)[0][1] for i in range(len(test_sequences))]
print(mode_counts)

# 최종 예측의 정확도를 계산합니다.
final_accuracy = sum([1 if prediction == truth else 0 for prediction, truth in zip(final_predictions, ground_truth_labels)]) / len(ground_truth_labels)

# 각 시퀀스에 대한 예측 확률의 평균을 계산합니다.
average_probabilities = [statistics.mean([probabilities[i] for probabilities in prob10]) for i in range(len(test_sequences))]

# ROC curve와 AUC를 계산합니다.
fpr, tpr, thresholds = roc_curve(y_test.cpu().numpy(), average_probabilities)
auc_score = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

print("test_sequences: ", test_sequences)
print("ground_truth_labels: ", ground_truth_labels)

# 예측 결과와 정확도 출력
print("Predictions: ", final_predictions)
print("Accuracy: ", final_accuracy)
print("probabilities: ", average_probabilities)

file_path = f'/content/drive/My Drive/Testdata/{finaldata_dir}/final_result_nnc.txt'  # 원하는 경로 및 파일명으로 변경 가능

with open(file_path, 'w') as file:
    for item in final_predictions:
        file.write(item + '\n')

